<a href="https://colab.research.google.com/github/Sakib635/sage2.0/blob/main/3rd_Copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install z3-solver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 MB 16.0 MB/s eta 0:00:00


In [98]:
import os
import json
import re
from z3 import Solver, Bool, Or, And, Implies, sat, Int, String, Not, Real, simplify, Optimize
import time
import logging

In [99]:
# Function to read the requirements.txt file from a directory
def read_requirements(directory):
    with open(os.path.join(directory, 'r3.txt'), 'r') as file:
        return file.read()

# Function to read the JSON file from a directory
def read_json_file(directory, filename='updated_formated_8k.json'):
    with open(os.path.join(directory, filename), 'r') as file:
        return json.load(file)


In [100]:
# Function to parse the requirements.txt content
def parse_requirements(requirements_txt):
    requirements = {}
    lines = requirements_txt.strip().split('\n')
    for line in lines:
        line = line.strip()
        if line:
            if '==' in line:
                package, version = line.split('==')
                requirements[package.strip()] = ('==', version.strip())
            elif '>=' in line:
                package, version = line.split('>=')
                requirements[package.strip()] = ('>=', version.strip())
            elif '<=' in line:
                package, version = line.split('<=')
                requirements[package.strip()] = ('<=', version.strip())
            elif '>' in line:
                package, version = line.split('>')
                requirements[package.strip()] = ('>', version.strip())
            elif '<' in line:
                package, version = line.split('<')
                requirements[package.strip()] = ('<', version.strip())
            elif '!=' in line:
                package, version = line.split('!=')
                requirements[package.strip()] = ('!=', version.strip())
    return requirements


In [101]:

# Function to fetch versions according to constraints
def fetch_versions(requirements, projects_data):
    matching_versions = {}
    for package, (operator, version_constraint) in requirements.items():
        if package in projects_data['projects']:
            available_versions = projects_data['projects'][package].keys()
            if operator == '==':
                pattern = re.compile(f'^{version_constraint.replace("*", ".*")}$')
                matching_versions[package] = [v for v in available_versions if pattern.match(v)]
            elif operator == '>=':
                matching_versions[package] = [v for v in available_versions if v >= version_constraint]
            elif operator == '<=':
                matching_versions[package] = [v for v in available_versions if v <= version_constraint]
            elif operator == '>':
                matching_versions[package] = [v for v in available_versions if v > version_constraint]
            elif operator == '<':
                matching_versions[package] = [v for v in available_versions if v < version_constraint]
            elif operator == '!=':
                matching_versions[package] = [v for v in available_versions if v != version_constraint]
    return matching_versions

# Function to fetch dependencies for a specific version of a package
def fetch_dependencies(package, version, projects_data):
    dependencies = projects_data['projects'][package][version].get('dependency_packages', [])
    if dependencies is None:
        dependencies = []
    print(f"Package: {package}, Version: {version}, Dependencies: {dependencies}")
    return dependencies



# Function to parse requirements from a list of dependencies
def parse_requirements_from_list(dependencies):
    requirements = {}
    for dependency in dependencies:
        print(f'hello-dependency: ',dependency)
        if '==' in dependency:
            package, version = dependency.split('==')
            requirements[package.strip()] = ('==', version.strip())

        elif '>=' in dependency:
            package, version = dependency.split('>=')
            requirements[package.strip()] = ('>=', version.strip())
        elif '<=' in dependency:
            package, version = dependency.split('<=')
            requirements[package.strip()] = ('<=', version.strip())
        elif '>' in dependency:
            package, version = dependency.split('>')
            requirements[package.strip()] = ('>', version.strip())
        elif '<' in dependency:
            package, version = dependency.split('<')
            requirements[package.strip()] = ('<', version.strip())
        elif '!=' in dependency:
            package, version = dependency.split('!=')
            requirements[package.strip()] = ('!=', version.strip())
    #print(f"requirements = {requirements}")
    return requirements



In [102]:
# Function to recursively fetch versions and their dependencies
def fetch_versions_and_dependencies(requirements, projects_data, visited=None):

    if visited is None:
        visited = set()
    direct_dependencies = fetch_versions(requirements, projects_data)
    transitive_dependencies = {}

    for package, versions in list(direct_dependencies.items()):  # Use list to create a copy for iteration
        for version in versions:
            if (package, version) not in visited:
                visited.add((package, version))
                dependencies = fetch_dependencies(package, version, projects_data)
                #todo: Is this correct
                if dependencies:  # Only proceed if there are dependencies
                    dependency_requirements = parse_requirements_from_list(dependencies)

                    if dependency_requirements:
                        dep_direct, dep_transitive = fetch_versions_and_dependencies(dependency_requirements, projects_data, visited)
                        transitive_dependencies[f"{package}=={version}"] = dep_direct
                        transitive_dependencies.update(dep_transitive)
    #print(f"Direct Dependencies generate_smt: {direct_dependencies}")
    #print(f"Transitive Dependencies generate_smt: {transitive_dependencies}")
    return direct_dependencies, transitive_dependencies

In [103]:
def generate_smt_expression(direct_dependencies, transitive_dependencies):
    # Initialize an Optimize solver to handle both hard and soft constraints
    solver = Optimize()
    constraints = []

    # Generate constraints for direct dependencies
    for package, versions in direct_dependencies.items():
        if isinstance(versions, list):
            # Create a constraint that the package version must be one of the specified versions
            package_constraint = Or([String(package) == v for v in versions])
            constraints.append(package_constraint)
            # Add soft constraints with weights for versions
            sorted_versions = sorted(versions, reverse=False)  # Sort versions to prioritize newer versions
            weight = 1
            for version in sorted_versions:
                # Add a soft constraint with increasing weight for newer versions
                solver.add_soft(String(package) == version, weight)
                weight += 1  # Increment the weight for the next version

    # Generate constraints for transitive dependencies
    for package_version, dependencies in transitive_dependencies.items():
        if isinstance(dependencies, dict):
            # Split the package_version to get the package name and its version
            package, version = package_version.split('==')
            for dep_package, dep_versions in dependencies.items():
                # Create a constraint for each dependency that it must be one of the specified versions
                dependency_constraint = Or([String(dep_package) == dep_version for dep_version in dep_versions])
                constraints.append(Implies(String(package) == version, dependency_constraint))
                # Add soft constraints with weights for versions
                sorted_versions = sorted(dep_versions, reverse=False)  # Sort versions to prioritize newer versions
                weight = 1
                for dep_version in sorted_versions:
                    # Add a soft constraint with increasing weight for newer versions
                    solver.add_soft(String(dep_package) == dep_version, weight)
                    weight += 1  # Increment the weight for the next version

    # Combine all constraints into a single final constraint
    final_constraint = And(constraints)
    solver.add(final_constraint)

    return solver, constraints


In [104]:
# Function to solve the SMT expression
def smt_solver(solver):
    # Check for the maximum satisfaction
    if solver.check() == sat:
        print("The expression is satisfiable with the maximum satisfaction of soft constraints.")
        model = solver.model()
        return {d.name(): model[d] for d in model.decls()}
    else:
        print("Not satisfiable.")
        return None


In [106]:

def main():
    directory = '/content/drive/MyDrive/smart pip sample data'



    # Log file setup
    log_file = 'execution_log.txt'

    def log_execution_time(action_name, start_time, end_time):
        with open(log_file, 'a') as file:
            file.write(f'{action_name} execution time: {end_time - start_time} seconds\n')



    # Read files from the directory
    start_time = time.time()
    requirements_txt = read_requirements(directory)
    projects_data = read_json_file(directory)
    end_time = time.time()
    log_execution_time("Reading files", start_time, end_time)



    # Parse requirements
    start_time = time.time()
    requirements = parse_requirements(requirements_txt)
    end_time = time.time()
    log_execution_time("Parsing requirements", start_time, end_time)
    print("Parsed requirements:", requirements)



    # Fetch matching versions and their dependencies
    start_time = time.time()
    direct_dependencies, transitive_dependencies = fetch_versions_and_dependencies(requirements, projects_data)
    end_time = time.time()
    log_execution_time("Fetching versions and dependencies", start_time, end_time)
    print("Direct dependencies:", direct_dependencies)
    print("Transitive dependencies:", transitive_dependencies)
    # Save to a file (optional)
    with open('Transitive_expression.json', 'w') as file:
        file.write(str(transitive_dependencies))



    # Generate SMT expression
    start_time = time.time()
    solver, constraints = generate_smt_expression(direct_dependencies, transitive_dependencies)
    end_time = time.time()
    log_execution_time("Generating SMT expression", start_time, end_time)
    # Save to a file (optional)
    with open('SMT_expression.txt', 'w') as file:
      file.write(str(solver))



    # Solve the SMT expression
    start_time = time.time()
    solution = smt_solver(solver)
    end_time = time.time()
    log_execution_time("Solving SMT expression", start_time, end_time)
    print(f'Possible Solutions: {solution}')

if __name__ == "__main__":
    main()

Parsed requirements: {'idna': ('>', '2.6'), 'requests': ('==', '2.22.0')}
Package: idna, Version: 3.7, Dependencies: []
Package: idna, Version: 3.6, Dependencies: []
Package: idna, Version: 3.5, Dependencies: []
Package: idna, Version: 3.4, Dependencies: []
Package: idna, Version: 3.3, Dependencies: []
Package: idna, Version: 3.2, Dependencies: []
Package: idna, Version: 3.1, Dependencies: []
Package: idna, Version: 3.0, Dependencies: []
Package: idna, Version: 2.9, Dependencies: []
Package: idna, Version: 2.8, Dependencies: []
Package: idna, Version: 2.7, Dependencies: []
Package: requests, Version: 2.22.0, Dependencies: ['chardet<3.1.0,>=3.0.2', 'idna<2.9,>=2.5', 'urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1', 'certifi>=2017.4.17']
hello-dependency:  chardet<3.1.0,>=3.0.2
hello-dependency:  idna<2.9,>=2.5
hello-dependency:  urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1
hello-dependency:  certifi>=2017.4.17
Package: certifi, Version: 2024.2.2, Dependencies: []
Package: certifi, Version: 2023.11.